In [1]:
import networkx as nx
import json

In [2]:
"""
Graph with CWEs, APs, techniques, tactics

Edges between CWE/APs and APs/techniques are such that every CWE/AP or AP/technique is represented in at least one of the edges, and all edges
have at least one link in the true graph. For example, in the current graph, for the AP node with count 22 is connected to technique nodes with 
counts 3, 83, and 24. The number of links in these four edges will add to at least 22 (for each AP), and each edge has at least one link.

Edges linked to tactics are such that every element in the technique node connects to at least one of the tactics.
"""

'\nGraph with CWEs, APs, techniques, tactics\n\nEdges between CWE/APs and APs/techniques are such that every CWE/AP or AP/technique is represented in at least one of the edges, and all edges\nhave at least one link in the true graph. For example, in the current graph, for the AP node with count 22 is connected to technique nodes with \ncounts 3, 83, and 24. The number of links in these four edges will add to at least 22 (for each AP), and each edge has at least one link.\n\nEdges linked to tactics are such that every element in the technique node connects to at least one of the tactics.\n'

In [3]:
with open("w_dict.json", "r") as f:
    w_dict = json.load(f)
with open("ap_dict.json", "r") as f:
    ap_dict = json.load(f)
with open("technique_dict.json", "r") as f:
    technique_dict = json.load(f)

f = open("cwe_mitigation_ids_temp.json")
w_mitigation = json.load(f)

f = open("capec_mitigation_temp.json")
ap_mitigation = json.load(f)

f = open("technique_mitigation_temp.json")
technique_mitigation = json.load(f)

f = open("technique_detection_temp.json")
technique_detection = json.load(f)

f = open("capec_detection_temp.json")
ap_detection = json.load(f)

f = open("cwe_detection_temp.json")
w_detection = json.load(f)

f = open("d3fend_technique.json")
d3fend_technique = json.load(f)

f = open("engage_technique.json")
engage_technique = json.load(f)

f = open("d3fend.json")
d3 = json.load(f)

f = open("engage_activity.json")
engage_activity = json.load(f)

f = open("tactic_temp.json")
tactic = json.load(f)

In [4]:
G = nx.Graph()

In [5]:
counts_dict = {}
for w in w_dict:
    links = w_dict[w]
    str = ["cwe"]
    if links["ap"]:
        str.append("aps")

    techniques = False
    no_techniques = False
    tactics = False
    no_tactics = False
    for ap in links["ap"]:
        if ap_dict[ap]["techniques"]:
            techniques = True
        else:
            no_techniques = True
        for technique in ap_dict[ap]["techniques"]:
            if technique_dict[technique]["tactics"]:
                tactics = True
            else:
                no_tactics = True
    if techniques:
        str.append("techniques")
    if no_techniques:
        str.append("no techniques")
    if tactics:
        str.append("tactics")
    if no_tactics:
        str.append("no tactics")

    str = "_".join(str)

    if str in counts_dict:
        counts_dict[str] += 1
    else:
        counts_dict[str] = 1

counts_dict

{'cwe': 600,
 'cwe_aps_no techniques': 256,
 'cwe_aps_techniques_no techniques_tactics': 55,
 'cwe_aps_techniques_tactics': 13}

In [6]:
for key in counts_dict:
    G.add_node(key, label=counts_dict[key], color="red", title=key)

In [7]:
counts_dict = {}
for ap in ap_dict:
    links = ap_dict[ap]
    str = ["ap"]
    if links["cwes"]:
        str.append("cwes")
    else:
        str.append("no cwes")

    if links["techniques"]:
        str.append("techniques")
    else:
        str.append("no techniques")

    tactics = False
    no_tactics = False
    for technique in links["techniques"]:
        if technique_dict[technique]["tactics"]:
            tactics = True
        else:
            no_tactics = True

    if tactics:
        str.append("tactics")
    if no_tactics:
        str.append("no tactics")

    str = "_".join(str)

    if str in counts_dict:
        counts_dict[str] += 1
    else:
        counts_dict[str] = 1

counts_dict

{'ap_cwes_no techniques': 332,
 'ap_cwes_techniques_tactics': 73,
 'ap_no cwes_techniques_tactics': 18,
 'ap_no cwes_no techniques': 123}

In [8]:
for key in counts_dict:
    G.add_node(key, label=counts_dict[key], color="green", title=key)
    true_key = key.split("_")
    if "cwes" in true_key:
        for node in G.nodes:
            if node.startswith("cwe"):
                true_node = node.split("_")
                if "techniques" in true_key and "techniques" in true_node:
                    tactics_set_key = set()
                    tactics_set_node = set()
                    for term in true_key:
                        if "tactics" in term:
                            tactics_set_key.add(term)
                    for term in true_node:
                        if "tactics" in term:
                            tactics_set_node.add(term)

                    if tactics_set_key == tactics_set_node:
                        G.add_edge(key, node, color="black")
                if "no techniques" in true_key and "no techniques" in true_node:
                    G.add_edge(key, node, color="black")

In [9]:
for technique in technique_dict:
    technique_dict[technique]["ap"] = []

for ap in ap_dict:
    for technique in ap_dict[ap]["techniques"]:
        technique_dict[technique]["ap"].append(ap)

In [10]:
counts_dict = {}
for technique in technique_dict:
    links = technique_dict[technique]
    str = ["technique"]
    if links["ap"]:
        str.append("aps")

    if links["tactics"]:
        str.append("tactics")
    else:
        str.append("no tactics")

    cwes = False
    no_cwes = False

    for ap in technique_dict[technique]["ap"]:
        if ap_dict[ap]["cwes"]:
            cwes = True
        else:
            no_cwes = True

    if cwes:
        str.append("cwes")
    if no_cwes:
        str.append("no cwes")

    str = "_".join(str)

    if str in counts_dict:
        counts_dict[str] += 1
    else:
        counts_dict[str] = 1

counts_dict

{'technique_tactics': 465,
 'technique_aps_tactics_cwes': 80,
 'technique_aps_tactics_no cwes': 18,
 'technique_aps_tactics_cwes_no cwes': 3}

In [11]:
tactics = set()

for technique in technique_dict:
    tactics.update(technique_dict[technique]["tactics"])

print(len(tactics))

G.add_node("tactics", label=len(tactics), color="blue", title="tactics")

14


In [12]:
for key in counts_dict:
    G.add_node(key, label=counts_dict[key], color="orange", title=key)
    true_key = key.split("_")
    if "tactics" in true_key:
        G.add_edge(key, "tactics", color="black")
    for node in G.nodes:
        true_node = node.split("_")
        if "ap" in true_node:
            if ("cwes" in true_node and "cwes" in true_key) or (
                "no cwes" in true_node and "no cwes" in true_key
            ):
                if ("tactics" in true_node and "tactics" in true_key) or (
                    "no tactics" in true_node and "no tactics" in true_key
                ):
                    G.add_edge(key, node, color="black")

In [13]:
# Save graph
file_path = "demo/complete_graph_networkx.pkl"
nx.write_gpickle(G, file_path)

In [14]:
from pyvis.network import Network

nt = Network("600px", "600px", notebook=True)
nt.from_nx(G)
nt.show("complete.html")